In [15]:
# Import packages
import os
import sys
import random
import pandas as pd
import torch
import torchaudio
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer
from datasets import Dataset, Audio
#from peft import LoraConfig, get_peft_model
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datetime import datetime

In [17]:
# Processing function
def prepare_dataset(batch):
    '''
    Process dataset for training
    '''
    audio = batch["audio"]
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0] # feature extractor
    batch["labels"] = processor(text=batch["text"]).input_ids # tokenizer
    return batch

# Data collator
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [18]:
# load processor
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# load data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [19]:
# Convert to dataset
df = pd.read_csv('training_data.csv')
dataset = Dataset.from_pandas(df)

# Clean structure for downstram preprocessing
dataset = dataset.rename_column("transcription", "text") 
dataset = dataset.rename_column("file_name", "audio") 
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Train-test split
dataset = dataset.train_test_split(test_size=0.2)

# Process data
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=1)

Map: 100%|██████████| 239/239 [00:00<00:00, 433.36 examples/s]


In [20]:
# Instantiate Wav2Vec2 model
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

# Freeze feature extractor layer
model.freeze_feature_extractor()

/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2175: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze

In [23]:
# Initialize training arguments
training_args = TrainingArguments(
  output_dir="/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/finetuned_w2v2",
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4, 
  eval_strategy="no",
  save_strategy='no',
  num_train_epochs=5,
  fp16=True,
  gradient_checkpointing=True, 
#   save_steps=500,
#   eval_steps=500,
#   logging_steps=10,
  learning_rate=5e-4,
  weight_decay=0.0,
  warmup_steps=1000,
  save_total_limit=2,
    label_names=['labels']
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor
)

/tmp/ipykernel_728669/2298923560.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
# Save training duration
start_time = datetime.now()
trainer.train()
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: 